In [1]:
import requests

Atributos del nodo:

1. Nombre (del conocimiento)
2. Literal (descripción)
3. Narrower concepts (conocimientos hijos)
4. Broader concepts (conocimientos ancestros)
5. Tipo de competencia (conocimiento, habilidad,...)
6. Competencia optativa para la ocupación x
7. Competencia esencial para la ocupación x
8. Nivel del conocimiento
9. Número de ocupaciones relacionadas al nodo
10. peso? (pensar si definir un peso de la arista es necesario)


----

Pasos:

1. Conectarse con la API de ESCO  **(Check)**
2. Comprender cuales son las llaves de la respuesta **(Check)**
3. Sacar un listado de todos los conocimientos con la API (mirar exceles) **(en proceso)**
4. Definir las clases para hacer el procedimiento de extracción y almacenamiento.
5. Crear el grafo.

-----

### Conectarse con la API

In [43]:
import requests

def get_esco_concept_data(uri: str, language: str = "es") -> dict:
    """
    Consulta la ESCO API para un recurso de tipo 'skill' o 'concept',
    y devuelve una versión filtrada con solo los datos esenciales.

    Args:
        uri (str): URI del recurso ESCO (por ejemplo, un skill o categoría).
        language (str): Idioma preferido (por defecto "es").

    Returns:
        dict: Diccionario con los datos esenciales o un error.
    """

    # Endpoint de ESCO (modifica a 'concept' si es necesario)
    url = "https://ec.europa.eu/esco/api/resource/skill"

    params = {
        "uri": uri,
        "language": language
    }

    headers = {
        "Accept": "application/json"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Claves de interés dentro de _links
        link_keys = [
            'hasSkillType', 'broaderConcept', 'broaderHierarchyConcept',
            'narrowerSkill', 'narrowerConcept',
            'isEssentialForOccupation', 'isOptionalForOccupation'
        ]

        # Extraer solo uri y title
        filtered_links = {
            key: [
                {
                    'uri': item.get('uri'),
                    'title': item.get('title')
                }
                for item in data.get('_links', {}).get(key, [])
            ]
            for key in link_keys
        }

        # Extraer ancestors con uri y título
        raw_ancestors = data.get('_embedded', {}).get('ancestors', [])
        filtered_ancestors = [
            {
                'uri': item.get('_links', {}).get('self', {}).get('uri'),
                'title': item.get('title')
            }
            for item in raw_ancestors
        ]

        # Compilar resultado esencial
        result = {
            "className": data.get("className"),
            "uri": data.get("uri"),
            "title": data.get("title"),
            "preferredLabel": data.get("preferredLabel", {}).get(language),
            "description": (
                data.get("description", {}).get(language, {}).get("literal")
                or data.get("description", {}).get("en", {}).get("literal")
            ),
            "_links": filtered_links,
            "_embedded": {
                "ancestors": filtered_ancestors,
                "title": data.get("_embedded", {}).get("title")
            }
        }

        return result

    else:
        return {
            "error": f"Error {response.status_code}: {response.text}",
            "uri": uri
        }


In [49]:
uri = "http://data.europa.eu/esco/skill/fed5b267-73fa-461d-9f69-827c78beb39d"
concept_info = get_esco_concept_data(uri, language="es")

for key, value in concept_info.items():
    print(f"{key}: {value}\n")

className: Skill

uri: http://data.europa.eu/esco/skill/fed5b267-73fa-461d-9f69-827c78beb39d

title: técnicas teatrales

preferredLabel: técnicas teatrales

description: Comprender las técnicas que facilitan la exitosa representación de una obra.

_links: {'hasSkillType': [{'uri': 'http://data.europa.eu/esco/skill-type/knowledge', 'title': 'conocimiento'}], 'broaderConcept': [], 'broaderHierarchyConcept': [{'uri': 'http://data.europa.eu/esco/isced-f/0215', 'title': 'música y artes escénicas'}], 'narrowerSkill': [], 'narrowerConcept': [], 'isEssentialForOccupation': [{'uri': 'http://data.europa.eu/esco/occupation/00030d09-2b3a-4efd-87cc-c4ea39d27c34', 'title': 'director técnico/directora técnica'}, {'uri': 'http://data.europa.eu/esco/occupation/07b91f36-8272-493a-8e1c-5625e77efe68', 'title': 'bailarín/bailarina'}, {'uri': 'http://data.europa.eu/esco/occupation/949f1dc9-fb19-4212-8ac6-35ce83c88921', 'title': 'apuntador/apuntadora'}, {'uri': 'http://data.europa.eu/esco/occupation/00747307

In [2]:
import requests

def get_knowledge_skills(language="es", limit=50, offset=0):
    """
    Fetch ESCO concepts from the 'skills' scheme and filter by skill type 'knowledge'.

    Returns:
        dict: {uri: title} of knowledge skills
    """
    url = "https://ec.europa.eu/esco/api/resource/skill"

    params = {
        "isInScheme": "http://data.europa.eu/esco/concept-scheme/skills",
        "isInSkillType": "http://data.europa.eu/esco/skill-type/knowledge",
        "language": language,
        "limit": limit,
        "offset": offset
    }

    headers = {
        "Accept": "application/json"
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return {}

    response_data = response.json()
    embedded = response_data.get('_embedded', {})
    knowledge_concepts = {}

    for concept_uri, concept_data in embedded.items():
        if not isinstance(concept_data, dict):
            continue

        skill_types = concept_data.get('_links', {}).get('hasSkillType', [])

        is_knowledge = any(
            link.get('uri') == 'http://data.europa.eu/esco/skill-type/knowledge'
            for link in skill_types
        )

        if is_knowledge:
            title = concept_data.get("title")
            if title:
                knowledge_concepts[concept_uri] = title

    return knowledge_concepts


In [6]:
all_knowledge_skills = {}
offset = 0
limit = 50

while True:
    batch = get_knowledge_skills(limit=limit, offset=offset)
    if not batch:
        break
    all_knowledge_skills.update(batch)
    offset += limit

In [7]:
len(all_knowledge_skills)

55

-----

In [8]:
import json

# After the while loop ends
with open("knowledge_skills.json", "w", encoding="utf-8") as f:
    json.dump(all_knowledge_skills, f, ensure_ascii=False, indent=2)

In [12]:
import pandas as pd

# Read the CSV with selected columns
data = pd.read_csv("occupationSkillRelations_es.csv", usecols=['skillUri', 'skillType'])

# Filter rows where skillType is 'knowledge'
knowledge_skills = data[data['skillType'] == 'knowledge'].drop_duplicates(subset="skillUri").reset_index(drop=True)



In [15]:
knowledge_skills['skillUri']

0       http://data.europa.eu/esco/skill/fed5b267-73fa...
1       http://data.europa.eu/esco/skill/3b3b7373-220a...
2       http://data.europa.eu/esco/skill/7f107a85-d6ac...
3       http://data.europa.eu/esco/skill/86ed2a45-eae9...
4       http://data.europa.eu/esco/skill/8d4271ca-c9fd...
                              ...                        
3142    http://data.europa.eu/esco/skill/6b4649e5-25a6...
3143    http://data.europa.eu/esco/skill/b5936963-0d12...
3144    http://data.europa.eu/esco/skill/3c1284d3-2602...
3145    http://data.europa.eu/esco/skill/2fd40b98-4b70...
3146    http://data.europa.eu/esco/skill/c3bb5700-a94d...
Name: skillUri, Length: 3147, dtype: object